In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data = pd.read_csv("raw.csv")

In [3]:
data.shape

(956, 15)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 956 entries, 0 to 955
Data columns (total 15 columns):
Unnamed: 0           956 non-null int64
Job Title            956 non-null object
Salary Estimate      956 non-null object
Job Description      956 non-null object
Rating               956 non-null float64
Company Name         956 non-null object
Location             956 non-null object
Headquarters         956 non-null object
Size                 956 non-null object
Founded              956 non-null int64
Type of ownership    956 non-null object
Industry             956 non-null object
Sector               956 non-null object
Revenue              956 non-null object
Competitors          956 non-null object
dtypes: float64(1), int64(2), object(12)
memory usage: 112.2+ KB


In [5]:
data.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\r\nLocation: Albuquerque, NM\r\...",3.8,Tecolote Research\r\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\r\n\r\nI. General Summary\r\...,3.4,University of Maryland Medical System\r\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\r\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\r\nJob ID: 310709\r...,3.8,PNNL\r\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\r\nAffinity Solutions / Marketi...,2.9,Affinity Solutions\r\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [6]:
#remove rows don't having salary estimate data
data=data[data["Salary Estimate"]!="-1"]

In [7]:
data.shape

(742, 15)

In [8]:
#salary parsing
salary = data["Salary Estimate"].apply(lambda x: re.findall(r'\d+', x))

#min-max salary
min_salary = salary.apply(lambda x: int(x[0]))
max_salary = salary.apply(lambda x: int(x[-1]))

#basis of paying
hourly = data["Salary Estimate"].apply(lambda x: 1 if "per hour" in x.lower() else 0)
employer_provided = data["Salary Estimate"].apply(lambda x: 1 if "employer provided" in x.lower() else 0)

#average salary
avg_salary = np.average(np.array([min_salary,max_salary]), axis=0)


#appending into dataframe
data = data.join(pd.DataFrame(
    {
        'min_salary': min_salary,
        'max_salary': max_salary,
        'avg_salary': avg_salary,
        'hourly': hourly,
        'employer_provided': employer_provided 
    }, index=data.index
))

#hourly to annualy
data.min_salary = data.apply(lambda x: x.min_salary*2 if x.hourly == 1 else x.min_salary, axis=1)
data.max_salary = data.apply(lambda x: x.max_salary*2 if x.hourly == 1 else x.max_salary, axis=1)
data.avg_salary = data.apply(lambda x: x.avg_salary*2 if x.hourly == 1 else x.avg_salary, axis=1)



In [9]:
#state field
job_state = data["Location"].apply(lambda x: x.split(",")[1])
same_state = data.apply(lambda x: 1 if x.Location == x.Headquarters else 0, axis = 1)


#appending into dataframe
data = data.join(pd.DataFrame(
    {
        'job_state': job_state,
        'same_state': same_state
    }, index=data.index
))

In [10]:
#age of company
age=data["Founded"].apply(lambda x: x if x ==-1 else (2020 - x))


#appending into dataframe
data['age'] = age

In [11]:
#job description parsing

#python
python_l = data["Job Description"].apply(lambda x: 1 if 'python' in x.lower() else 0)

#r studio
r_st = data["Job Description"].apply(lambda x: 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)

#sas
sas = data["Job Description"].apply(lambda x: 1 if 'sas' in x.lower() else 0)

#spark
spark = data["Job Description"].apply(lambda x: 1 if 'spark' in x.lower() else 0)

#aws
aws = data["Job Description"].apply(lambda x: 1 if 'aws' in x.lower() else 0)

#sql
sql = data["Job Description"].apply(lambda x: 1 if 'sql' in x.lower() else 0)

#excel
exc = data["Job Description"].apply(lambda x: 1 if 'excel' in x.lower() else 0)

#matlab
mat = data["Job Description"].apply(lambda x: 1 if 'matlab' in x.lower() else 0)

#description length
desc_len = data['Job Description'].apply(lambda x: len(x))


#appending extracted data into dataframe
data = data.join(pd.DataFrame(
    {
        'python' : python_l,
        'r' : r_st,
        'sas' : sas,
        'spark' : spark,
        'aws' : aws,
        'sql' : sql,
        'excel' : exc,
        'matlab' : mat
    }, index=data.index
))

In [12]:
data['desc_len'] = data['Job Description'].apply(lambda x: len(x))


In [14]:
#job title designation

#level-1 consist of junior, jr, jr.
level1= np.array(data["Job Description"].apply(lambda x: "junior" if "junior" in x.lower() or "jr" in x.lower() or "jr." in x.lower() else "other"))

#level-2 consist of senior, sr, sr., lead
level2 = np.array(data["Job Description"].apply(lambda x: "senior" if "senior" in x.lower() or "sr" in x.lower() or "sr." in x.lower() or "lead" in x.lower() or "manager" in x.lower() else "other"))

#level-3 consist of director, principal, vp
level3 = np.array(data["Job Description"].apply(lambda x: "higher" if "director" in x.lower() or "principal" in x.lower() or "vp" in x.lower() else "other"))



level=[]
for i in range(len(level3)):
    if level3[i]=="higher":
        level.append("higher")
    elif level2[i]=="senior":
        level.append("senior")
    elif level1[i]=="junior":
        level.append("junior")
    else:
        level.append("other")
        

#appending extracted level data into dataframe
data["level"] = level
        

In [15]:
data.level.value_counts()

senior    442
other     154
higher    137
junior      9
Name: level, dtype: int64

In [16]:
# job title

#separate column according to job title as data scientist, data engineer and data analyst
title= data["Job Description"].apply(lambda x: "DS" if "data scientist" in x.lower() else "DE" if "data engineer" in x.lower() else "A" if "analyst" in x.lower() else "mle" if "machine learning" in x.lower() else "other")


#appending into dataframe
data["title"] = title

data.title.value_counts()

DS       308
other    183
A        105
DE        81
mle       65
Name: title, dtype: int64

In [17]:
#number of competitors
comp = data['Competitors'].apply(lambda x: len(x.split(',')) if x!="-1" else 0)


#appending
data["comp"] = comp

In [18]:
#name of company
name = data['Company Name'].apply(lambda x: x.split('\r')[0])

#appending
data['name'] = name

In [19]:
# Fix state Los Angeles 
data['job_state']= data.job_state.apply(lambda x: x.strip() if x.strip().lower() != 'los angeles' else 'CA')
data.job_state.value_counts()

CA    152
MA    103
NY     72
VA     41
IL     40
MD     35
PA     33
TX     28
NC     21
WA     21
NJ     17
FL     16
OH     14
TN     13
CO     11
DC     11
UT     10
WI     10
IN     10
MO      9
AZ      9
AL      8
MI      6
DE      6
KY      6
GA      6
IA      5
CT      5
LA      4
NE      4
OR      4
KS      3
NM      3
ID      2
MN      2
RI      1
SC      1
Name: job_state, dtype: int64

In [20]:
data.to_csv("cleaned_data.csv", index = False)